In [1]:
#Collap Drive Bağlama
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#Gerekli Kütüphaneler
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import word_tokenize
import joblib
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
#Train ve Test Veri Setleri Kurulumu

path = "/content/drive/MyDrive/Binovist  Sentiment Analysis/data/Hepsiburada Dataset 492k(%10 test)/"

df_train_filtered = pd.read_csv(path + "df_train_filtered.csv")
df_test_filtered = pd.read_csv(path + "df_test_filtered.csv")

dataframes_filtered = {'df_train': df_train_filtered, 'df_test': df_test_filtered}

yes_responses = {'evet', 'yes', 'y', '1', 'tamam', 'onay', 'olur', 'e'}
no_responses = {'hayır', 'no', 'n', '0', 'hayir', 'olmaz', 'h'}

#Eğer Temizleme Sonrası Boş Veri Olursa Çıkarma İşlemi
for name, df in dataframes_filtered.items():
  dataframes_filtered[name] = df.dropna()

In [4]:
# Oluştururalan yeni temiz verinin veri setine ekstra bir kolonlar'ı
df_train_filtered = dataframes_filtered['df_train']
df_test_filtered = dataframes_filtered['df_test']

X_train=df_train_filtered["processed_text"]
X_test=df_test_filtered["processed_text"]
y_train=df_train_filtered["label_numeric"]
y_test=df_test_filtered["label_numeric"]

print("x_train",X_train.shape)
print("x_test",X_test.shape)
print("y_train",y_train.shape)
print("y_test",y_test.shape)

x_train (152609,)
x_test (48951,)
y_train (152609,)
y_test (48951,)


In [5]:
print(X_train.isnull().sum())
print(X_test.isnull().sum())
print(y_train.isnull().sum())
print(y_test.isnull().sum())

0
0
0
0


In [6]:
# TfidfVectorizer kullanarak metinleri sayısal forma dönüştürme
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train).toarray()

In [ ]:
# Normalizasyon işlemi
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

In [ ]:
# Tensor'a dönüştürme
X_train = torch.tensor(X_train, dtype=torch.float32)

print(X_train)

In [ ]:
# 3'lü sınıflandırma için Linear SVM modeli
class LinearSVM(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(LinearSVM, self).__init__()
        self.linear = nn.Linear(input_dim, num_classes)  # Çıkış boyutu: 3 sınıf

    def forward(self, x):
        return self.linear(x)

# Hinge loss fonksiyonu (çoklu sınıflandırma için)
class MultiClassHingeLoss(nn.Module):
    def __init__(self):
        super(MultiClassHingeLoss, self).__init__()

    def forward(self, outputs, targets):
        targets_one_hot = torch.eye(outputs.shape[1])[targets].to(outputs.device)
        hinge_loss = torch.mean(torch.clamp(1 - (outputs - targets_one_hot), min=0))
        return hinge_loss

In [ ]:
# Model, loss ve optimizer tanımlama
input_dim = X_train.shape[1]
num_classes = 3  # Pozitif, nötr, negatif
model = LinearSVM(input_dim, num_classes)
criterion = MultiClassHingeLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)


In [ ]:
# Modelin eğitimi
num_epochs = 10
for epoch in range(num_epochs):
    for data, labels in train_loader:
        optimizer.zero_grad()

        # İleri yayılım
        outputs = model(data)

        # Loss hesaplama
        loss = criterion(outputs, labels)

        # Geri yayılım ve optimizasyon
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

print("Eğitim tamamlandı!")